In [1]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 16.9 MB/s eta 0:00:00


In [2]:
!pip install grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00


In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class State(TypedDict):
    input: str
    state: str

def even(state: State):
    print(f"even : {state['input']}")
    return {"input": state["input"], "state": "even" }

def odd(state: State):
    print(f"odd  :  {state['input']}")
    return {"input": state["input"], "state": "even"}

def even_step(state: State):
    print(f"even_step : {state['input'][1:]}")
    return {"input": state["input"][1:], "state": "even_step"}

def odd_step(state: State):
    print(f"odd_step  :  {state['input'][1:]}")
    return {"input": state["input"][1:], "state": "odd_step"}

def accept(state: State):
    print(f"accept  :  {state['input']}")
    return {"input": "", "state": "accept"}

def reject(state: State):
    print(f"reject  :  {state['input']}")
    return {"input": "", "state": "reject"}

def routing_even(state: State):
    print(f"routing_even :  {state['input']}")

    if state["input"] == "":
        return "accept"

    crr_char = state["input"][0]

    if crr_char == "1":
        return "odd_step"
    elif crr_char == "0":
        return "even_step"
    else:
        return "reject"

def routing_odd(state: State):
    print(f"routing_even :  {state['input']}")

    if state["input"] == "":
        return "reject"

    crr_char = state["input"][0]

    if crr_char == "1":
        return "even_step"
    elif crr_char == "0":
        return "odd_step"
    else:
        return "reject"

graph_builder = StateGraph(State)

graph_builder.add_node("odd",       odd)
graph_builder.add_node("even",      even)
graph_builder.add_node("odd_step",  odd_step)
graph_builder.add_node("even_step", even_step)
graph_builder.add_node("accept",    accept)
graph_builder.add_node("reject",    reject)

graph_builder.set_entry_point("odd")
graph_builder.set_finish_point("accept")
graph_builder.set_finish_point("reject")

graph_builder.add_conditional_edges(
    source="even",
    path=routing_even,
    path_map={"accept": "accept", "reject": "reject", "even_step": "even_step", "odd_step": "odd_step"},
)
graph_builder.add_conditional_edges(
    source="odd",
    path=routing_odd,
    path_map={"reject": "reject", "even_step": "even_step", "odd_step": "odd_step"},
)
graph_builder.add_edge("even_step", "even")
graph_builder.add_edge("odd_step", "odd")

graph = graph_builder.compile()

In [4]:
graph.get_graph().print_ascii()

                            +-----------+                           
                            | __start__ |                           
                            +-----------+                           
                                  *                                 
                                  *                                 
                                  *                                 
                              +-----+                               
                             .| odd |...                            
                        ..... +-----+** .....                       
                   .....      .        **    .....                  
              .....          .           *        .....             
           ...             ..             **           .....        
+-----------+             .                 *               ...     
| even_step |             .                 *                 .     
+-----------+             .       

In [5]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	odd(odd)
	even(even)
	odd_step(odd_step)
	even_step(even_step)
	accept(accept)
	reject(reject)
	__end__([<p>__end__</p>]):::last
	__start__ --> odd;
	even -.-> accept;
	even -.-> even_step;
	even -.-> odd_step;
	even -.-> reject;
	even_step --> even;
	odd -.-> even_step;
	odd -.-> odd_step;
	odd -.-> reject;
	odd_step --> odd;
	accept --> __end__;
	reject --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [6]:
graph.invoke({"input": "11"})

odd  :  11
routing_even :  11
even_step : 1
even : 1
routing_even :  1
odd_step  :  
odd  :  
routing_even :  
reject  :  


{'input': '', 'state': 'reject'}

In [7]:
graph.invoke({"input": "1"})

odd  :  1
routing_even :  1
even_step : 
even : 
routing_even :  
accept  :  


{'input': '', 'state': 'accept'}

In [8]:
graph.invoke({"input": "11111"})

odd  :  11111
routing_even :  11111
even_step : 1111
even : 1111
routing_even :  1111
odd_step  :  111
odd  :  111
routing_even :  111
even_step : 11
even : 11
routing_even :  11
odd_step  :  1
odd  :  1
routing_even :  1
even_step : 
even : 
routing_even :  
accept  :  


{'input': '', 'state': 'accept'}

In [9]:
graph.invoke({"input": "1a"})

odd  :  1a
routing_even :  1a
even_step : a
even : a
routing_even :  a
reject  :  a


{'input': '', 'state': 'reject'}